In [2]:
# Sample S&P program for one scrip

# STATUS: Completed
# Run-time: 1.3 mins

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
# ib = IB().connect('127.0.0.1', 7497, clientId=9) # rkv tws live
ib = IB().connect('127.0.0.1', 4001, clientId=9) # rkv IBG live

In [3]:
%%time
import numpy as np
import pandas as pd
from itertools import product
import datetime
import re

#******   Error catch in list comprehension  ****
#________________________________________________

def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        np.nan

# sd multiple for band
sigma = 2    # 2 sigma is about 95% probability
premium = 1.8    # e.g. 1.8 is 80% premium above

minHzn = 20   # minimum option horizon
maxHzn = 90   # maximum option horizon

base = 0.01   # Upper or Lower base multiple for prices

# market
exchange = 'SMART'
currency = 'USD'

# ... build the snp list

sym_chg_dict = {'BRK.B': 'BRK B', 'BRK/B': 'BRK B'} # Remap symbols in line with IBKR

snpurl = 'https://en.wikipedia.org/wiki/S%26P_100'
df_snp = pd.read_html(snpurl, header=0)[2]

df_snp.Symbol = df_snp.Symbol.map(sym_chg_dict).fillna(df_snp.Symbol)
df_snp['Type'] = 'Stock'

# Download cboe weeklies to a dataframe
dls = "http://www.cboe.com/publish/weelkysmf/weeklysmf.xls"

# read from row no 11, dropna and reset index
df_cboe = pd.read_excel(dls, header=11, 
                        usecols=[0,2,3]).loc[11:, :]\
                        .dropna(axis=0)\
                        .reset_index(drop=True)

# remove column names white-spaces and remap to IBKR
df_cboe.columns = df_cboe.columns.str.replace(' ', '')
df_cboe.Ticker = df_cboe.Ticker.map(sym_chg_dict).fillna(df_cboe.Ticker)

# list the equities
equities = [e for e in list(df_snp.Symbol) if e in list(df_cboe.Ticker)]

# filter and list the etfs
df_etf = df_cboe[df_cboe.ProductType == 'ETF'].reset_index(drop=True)
etfs = list(df_etf.Ticker)

# list the indexes
indexes = 'OEX,XEO,XSP,DJX'.split(',')

# Build a list of contracts
ss = [Stock(symbol=s, currency=currency, exchange=exchange) for s in set(equities+etfs)]
ixs = [Index(symbol=s,currency=currency, exchange='CBOE') for s in set(indexes)]
cs = ss+ixs

qcs = ib.qualifyContracts(*cs) # qualified underlyings

#****           Single scrip check. To be DELETED in function          *****
#...........................................................................
contract = next(q for q in qcs if q.symbol=='CVX')  # one symbol logic check
#___________________________________________________________________________

# Get the symbol
symbol = contract.symbol

#... Get volatility, hi52 and lo52
duration = '12 M'
size = '1 day'
bars = ib.reqHistoricalData(contract=contract, endDateTime='', 
                     durationStr=duration, barSizeSetting=size, 
                     whatToShow='TRADES', useRTH=True, 
                     formatDate=1, keepUpToDate=True)

stDev = np.std(a=[b.close for b in bars], ddof=0)

hi52 = max([b.high for b in bars])
lo52 = min([b.low for b in bars])

meanPrice = np.mean([b.close for b in bars])

# Get the underlying's price
ut = ib.reqTickers(contract)
# ib.sleep(0.01)
if  np.isnan(next(p.last for p in ut)):
    undPrice = next(p.close for p in ut)
else:
    undPrice = next(p.last for p in ut)

#... Get the option chains
chains = ib.reqSecDefOptParams(underlyingSymbol=contract.symbol, 
                      futFopExchange='', 
                      underlyingConId=contract.conId, underlyingSecType=contract.secType)

chain = next(c for c in chains if c.exchange=='SMART')

strikes = sorted(s for c in chains for s in c.strikes if (c.exchange == 'SMART') & 
                 ((s < undPrice - stDev*sigma) | (s > undPrice + stDev*sigma)))

expirations = sorted(exp for exp in chain.expirations 
                     if minHzn < (util.parseIBDatetime(exp)-datetime.datetime.now().date()).days < maxHzn)

rights = ['P', 'C']

#... Build and qualify the contracts
contracts = [Option(symbol, expiration, strike, right, 'SMART')
             for right in rights
             for expiration in expirations
             for strike in strikes]

ib.qualifyContracts(*contracts)

qual_contracts = [c for c in contracts if c.conId]

# remove unnecessary Puts and Calls
tgts = [t for t in qual_contracts 
       if (t.strike < undPrice - stDev*sigma) & (t.right == 'P') | 
          (t.strike > undPrice + stDev*sigma) & (t.right == 'C')]

tickers = [t for i in range(0, len(tgts), 20) for t in ib.reqTickers(*tgts[i:i + 20])]  # Get the tickers
ib.sleep(2)
tickers = [t for i in range(0, len(tgts), 20) for t in ib.reqTickers(*tgts[i:i + 20])]  # Get the tickers

lib_t = {t: catch(lambda: t.modelGreeks.undPrice) for t in tickers}
und_t = [k for k, v in lib_t.items() if v is not None]

# ... Build the dataframe
ts = [(t.contract.conId, t.contract.symbol, t.contract.lastTradeDateOrContractMonth, t.contract.strike, t.contract.right, float(t.contract.multiplier), 
       t.modelGreeks.undPrice, t.contract.localSymbol, t.bid, t.bidSize, t.ask, t.askSize, t.close, t.modelGreeks.impliedVol, t.modelGreeks.delta, 
       t.modelGreeks.optPrice, t.modelGreeks.pvDividend, t.modelGreeks.gamma, t.modelGreeks.vega, t.modelGreeks.theta, hi52, lo52, meanPrice, stDev, t) 
      for t in und_t]

cols = ['conId', 'ibSymbol', 'expiry', 'strike', 'right', 'lot', 'undPrice', 
        'localSymbol', 'bid', 'bidSize', 'ask', 'askSize', 'close', 'impliedVol', 'delta', 'optPrice', 
        'pvDividend', 'gamma', 'vega', 'theta', 'undHi', 'undLo', 'undMean', 'stdev', 'ticker']
df1 = pd.DataFrame(ts, columns=cols).sort_values(by=['expiry', 'strike'], ascending=False).reset_index(drop=True)

# get the margin
order = Order(action='SELL', totalQuantity=1, orderType='MKT')
whatif = [ib.whatIfOrder(contract, order) for contract in [t.contract for t in und_t]]
margin = pd.to_numeric([w.initMarginChange for w in whatif])

df1['whatif'] = whatif
df1 ['margin'] = margin

df1['expPrice'] = round(df1[['bid', 'ask', 'close']].max(axis=1)*premium/base)*base

df1['dte'] = (pd.to_datetime(df1.expiry) - datetime.datetime.now()).dt.days

# make negative dtes to 1 to accommodate last day option expiries
df1.loc[df1.dte <= 0, 'dte'] = 1

# calculate the rom
df1['rom'] = (df1.expPrice*df1.lot)/df1.margin*252/df1.dte

# remove calls with strike prices below undPrice and puts with strike prices above undPrice if any
# and sort it by expiry and strike
df = df1[~(((df1.strike < df1.undPrice) & (df1.right == 'C')) | ((df1.strike > df1.undPrice) & (df1.right == 'P')))].\
sort_values(by=['expiry', 'strike'], ascending=[True, False]).reset_index(drop=True)

# Pickle the dataframe if it is not empty
if not df.empty:
    df.to_pickle('./zdata/pkls/'+symbol+'.pkl')

Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Error 200, reqId 1485: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190125', strike=55.0, right='P', exchange='SMART')
Error 200, reqId 1486: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190125', strike=60.0, right='P', exchange='SMART')
Error 200, reqId 1487: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190125', strike=65.0, right='P', exchange='SMART')
Error 200, reqId 1488: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190125', strike=70.0, right='P', exchange='SMART')
Error 200, reqId 1489: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='201901

Error 200, reqId 1561: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190208', strike=65.0, right='P', exchange='SMART')
Error 200, reqId 1562: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190208', strike=70.0, right='P', exchange='SMART')
Error 200, reqId 1563: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190208', strike=75.0, right='P', exchange='SMART')
Error 200, reqId 1564: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190208', strike=80.0, right='P', exchange='SMART')
Error 200, reqId 1565: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190208', strike=85.0, right='P', exchange='SMART')
Error 200, reqId 1566: No security defin

Error 200, reqId 1617: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190215', strike=131.0, right='P', exchange='SMART')
Error 200, reqId 1618: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190215', strike=132.0, right='P', exchange='SMART')
Error 200, reqId 1619: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190215', strike=133.0, right='P', exchange='SMART')
Error 200, reqId 1624: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190215', strike=155.0, right='P', exchange='SMART')
Error 200, reqId 1625: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190215', strike=160.0, right='P', exchange='SMART')
Error 200, reqId 1626: No security 

Error 200, reqId 1697: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190125', strike=150.0, right='C', exchange='SMART')
Error 200, reqId 1698: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190125', strike=155.0, right='C', exchange='SMART')
Error 200, reqId 1699: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190125', strike=160.0, right='C', exchange='SMART')
Error 200, reqId 1700: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190125', strike=165.0, right='C', exchange='SMART')
Error 200, reqId 1701: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190125', strike=170.0, right='C', exchange='SMART')
Error 200, reqId 1702: No security 

Error 200, reqId 1757: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190208', strike=123.0, right='C', exchange='SMART')
Error 200, reqId 1758: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190208', strike=124.0, right='C', exchange='SMART')
Error 200, reqId 1759: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190208', strike=125.0, right='C', exchange='SMART')
Error 200, reqId 1760: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190208', strike=126.0, right='C', exchange='SMART')
Error 200, reqId 1761: No security definition has been found for the request, contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190208', strike=127.0, right='C', exchange='SMART')
Error 200, reqId 1762: No security 

...

Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190201', strike=135.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190201', strike=140.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190201', strike=145.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190201', strike=150.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190201', strike=155.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190201', strike=160.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190201', strike=165.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190201', strike=170.0, right='P', exchange='SMART')
Unknown contract

Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190215', strike=185.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190215', strike=190.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190215', strike=195.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190315', strike=55.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190315', strike=60.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190315', strike=65.0, right='P', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190315', strike=95.5, right='P', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190315', strike=96.0, right='P', exchange='SMART')
Unknown contract: Opt

Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190208', strike=60.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190208', strike=65.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190208', strike=70.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190208', strike=75.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190208', strike=80.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190208', strike=85.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190208', strike=90.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190208', strike=95.0, right='C', exchange='SMART')
Unknown contract: Option

Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190315', strike=128.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190315', strike=129.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190315', strike=131.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190315', strike=132.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190315', strike=133.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190315', strike=170.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190315', strike=175.0, right='C', exchange='SMART')
Unknown contract: Option(symbol='CVX', lastTradeDateOrContractMonth='20190315', strike=180.0, right='C', exchange='SMART')
Unknown contract

Wall time: 3min 14s


In [4]:
df1.loc[:, ['ibSymbol', 'right', 'strike', 'expiry', 'undPrice', 'close', 'margin', 'dte', 'stdev', 'undLo', 'undHi', 'undMean', 'bid', 'ask', 'expPrice', 'rom']]

,ibSymbol,right,strike,expiry,undPrice,close,margin,dte,stdev,undLo,undHi,undMean,bid,ask,expPrice,rom
0,CVX,C,165.0,20190315,109.236412,0.00,741.73,73,6.169939,100.22,133.88,120.247952,-1.0,-1.0,0.00,0.000000
1,CVX,C,160.0,20190315,109.236412,0.00,783.03,73,6.169939,100.22,133.88,120.247952,-1.0,-1.0,0.00,0.000000
2,CVX,C,155.0,20190315,109.236412,0.01,918.45,73,6.169939,100.22,133.88,120.247952,-1.0,-1.0,0.02,0.007517
3,CVX,C,150.0,20190315,109.236412,0.02,955.44,73,6.169939,100.22,133.88,120.247952,-1.0,-1.0,0.04,0.014452
4,CVX,C,145.0,20190315,109.236412,0.04,622.79,73,6.169939,100.22,133.88,120.247952,-1.0,-1.0,0.07,0.038800
5,CVX,P,95.0,20190315,109.227501,1.87,755.40,73,6.169939,100.22,133.88,120.247952,-1.0,-1.0,3.37,1.540035
6,CVX,P,90.0,20190315,109.227501,1.21,941.63,73,6.169939,100.22,133.88,120.247952,-1.0,-1.0,2.18,0.799197
7,CVX,P,85.0,20190315,109.227501,0.67,980.43,73,6.169939,100.22,133.88,120.247952,-1.0,-1.0,1.21,0.426036
8,CVX,P,80.0,20190315,109.227501,0.40,976.14,73,6.169939,100.22,133.88,120.247952,-1.0,-1.0,0.72,0.254623
9,CVX,P,75.0,20190315,109.227501,0.27,989.86,73,6.169939,100.22,133.88,120.247952,-1.0,-1.0,0.49,0.170883


In [ ]:
contract